# Tests on the JSSP

## Imports

In [1]:
import torch
from tensordict import TensorDict
from rl4co.envs.scheduling.jssp import JSSPEnv

Test the JSSP on the OR-tools instance: https://developers.google.com/optimization/scheduling/job_shop


In [2]:
low = 1
high = 99
n_j = 3
n_m = 3
torch.manual_seed(8)
env = JSSPEnv(n_j, n_m)
durations = torch.tensor([[3, 2, 2], [2, 1, 4], [4, 3, 0]], dtype=torch.float32)
machines = torch.tensor([[1, 2, 3], [1, 3, 2], [2, 3, 1]], dtype=torch.int32)
data = TensorDict(
    {"durations": durations.unsqueeze(0), "machines": machines.unsqueeze(0)},
    batch_size=1,
)
ortools_sol = [0, 1, 2, 0, 1, 1, 0, 2, 2]
env.reset(data)
total_reward = 0
for action in ortools_sol:
    data["action"] = torch.tensor([action], dtype=torch.long)
    td = env._step(data)
    total_reward += td["reward"].item()

# env.render()
make_span = env.initial_quality - total_reward
assert make_span.item() == 11

## Test with parallel envs

In [3]:
from torchrl.envs.vec_envs import ParallelEnv
envs = ParallelEnv(2, lambda: JSSPEnv(3, 3, _torchrl_mode=True))
td = envs.reset()
td["action"] = torch.tensor([[0], [1]])
td = envs.step(td)

assert torch.allclose(td["next"]["feasible_actions"], torch.tensor([[[1, 3, 6]], [[0, 4, 6]]]))

td["action"] = torch.tensor([[0], [1]])
td = envs.step(td)
assert torch.allclose(td["next"]["feasible_actions"], torch.tensor([[[2, 3, 6]], [[0, 5, 6]]]))